### SIERPIEŃ

In [3]:
import pandas as pd

In [9]:
url = 'https://raw.githubusercontent.com/kvmilos/OREX/main/kontrahenci.csv'
df = pd.read_csv(url, dtype=str)
dic = dict(zip('201-2-1-' + df['pozycja'], df['kod']))

def get_reservation(x: str):
    if x in dic:
        return dic[x]
    else:
        return x

In [10]:
samo = pd.read_excel('samo.xlsx', sheet_name='SAMO', header=1)
samo['reservation'] = samo['reservation'].astype(str)

In [24]:
samo = samo[samo['MONTH']=='Ağustos']

In [ ]:
samo.head()

In [88]:
df703 = pd.read_excel('symfonia.xlsx', sheet_name='703', header=None)
df704 = pd.read_excel('symfonia.xlsx', sheet_name='704', header=None)
df763 = pd.read_excel('symfonia.xlsx', sheet_name='763-7', header=None)

df703['source'] = '703'
df704['source'] = '704'
df763['source'] = '763-7'

symfonia = pd.concat([df703, df704, df763], ignore_index=True)

In [89]:
symfonia.columns = ['c1', 'c2', 'c3', 'c4', 'data', 'dokument', 'opis', 'c8', 'kwota', 'konto', 'c10', 'zrodlo']
symfonia['rezerwacja'] = symfonia['konto'].apply(get_reservation)

In [90]:
symfonia2 = symfonia.copy()

In [91]:
how = pd.read_excel('slownik.xlsx', header=0)

In [92]:
def change_reservation(x: str, how):
    if x in how['from'].values:
        return str(how[how['from'] == x]['to'].values[0])
    else:
        return x

symfonia2['rezerwacja'] = symfonia2['rezerwacja'].apply(lambda x: x if len(x) <= 7 else change_reservation(x, how))


In [93]:
symfonia2.head()

,c1,c2,c3,c4,data,dokument,opis,c8,kwota,konto,c10,zrodlo,rezerwacja
0,1,1,0,VIII,2023-08-01,2023/08/00001/R,rez. 1067853 Wczasy - TURCJA - Termin 25/07-01...,NaN,8745.40,201-2-1-51766,0,703,1067853
1,1,1,0,VIII,2023-08-01,2023/08/00002/R,Nr rezerwacji: 1013237 Pobyt zorganizowany - T...,NaN,14068.03,201-2-1-13492,0,703,1013237
2,1,1,0,VIII,2023-08-01,2023/08/00003/R,"Nr rezerwacji: 1019211 Wczasy - EGIPT, termin:...",NaN,3208.44,201-2-1-18787,0,703,1019211
3,1,1,0,VIII,2023-08-01,2023/08/00004/R,"Nr rezerwacji: 1019211 Wczasy - EGIPT, termin:...",NaN,3208.43,201-2-1-18787,0,703,1019211
4,1,1,0,VIII,2023-08-01,2023/08/00011/R,"Rez. 1025559 Usługa turystyczna - TURCJA, term...",NaN,6366.42,201-2-1-23925,0,703,1025559


In [94]:
symfonia2[symfonia2['rezerwacja'].str.len() > 7]['rezerwacja'].unique()

array(['201-2-1-3', '203-2-1-3', '203-2-1-286', '201-2-1-286',
       '203-2-1-65', '203-2-1-904'], dtype=object)

In [95]:
result = pd.DataFrame(columns=['res', 'sum_symfonia', 'SAMO_Cat_Price', 'difference'])
result['res'] = pd.concat([symfonia2['rezerwacja'], samo['reservation']]).unique()
result['sum_symfonia'] = result['res'].apply(lambda x: symfonia2[symfonia2['rezerwacja'] == x]['kwota'].astype(float).sum())
result['SAMO_Cat_Price'] = result['res'].apply(lambda x: samo[samo['reservation'] == x]['CataloguePrice'].astype(float).sum())
result['difference'] = (result['sum_symfonia'] - result['SAMO_Cat_Price']).round(2)
result['abs_difference'] = result['difference'].abs()
result['partner'] = result['res'].apply(lambda x: samo[samo['reservation'] == x]['Partner'].unique()[0] if len(samo[samo['reservation'] == x]['Partner'].unique()) > 0 else '')
result['zrodlo'] = result['res'].apply(lambda x: symfonia2[symfonia2['rezerwacja'] == x]['zrodlo'].unique()[0] if len(symfonia2[symfonia2['rezerwacja'] == x]['zrodlo'].unique()) > 0 else '')


In [96]:
result.head()

,res,sum_symfonia,SAMO_Cat_Price,difference,abs_difference,partner,zrodlo
0,1067853,8745.40,8745.0,0.40,0.40,WAKACJE.PL OPINOGORA GORNA,703
1,1013237,14068.03,14068.0,0.03,0.03,BIURO TURYSTYCZNE TURBACZ ANNA HARAS,703
2,1019211,6416.87,6354.0,62.87,62.87,FOSTERTRAVEL.PL - CENTRALA,703
3,1025559,6366.42,6366.0,0.42,0.42,BLIZEJ SLONCA S.C.,703
4,1081713,12960.00,12960.0,0.00,0.00,WAKACJE.PL RYBNIK CENTRUM,703


In [97]:
result.to_excel('result.xlsx', index=False)

### DMCC

In [2]:
import pandas as pd

In [4]:
dmcc_sergan = pd.read_excel('dmcc.xlsx', sheet_name='Plik DMCC_od Sergan', header=0)
dmcc_sergan['Reservation'] = dmcc_sergan['Reservation'].astype(str)

In [5]:
dmcc_sergan.head()

,Reservation,Catalogue price SAMO EUR,Tour start date,Tour ending,Reservation type,Tour,Hotel's partner,Included into black list:,Reason of including into black list,Partner's town,...,Net price,Profit,Paid,Debt,$,User,Author,Notes for incoming partner,Notes from incoming partner,Flight (price)
0,1151211,224,2023-08-06,2023-08-06,Only tickets,RZESZOW-ANTALYA ( Only Ticket ),NaN,No,NaN,Warszawa,...,0,224,0,224,EUR,n_svitlana,n_svitlana,"OW/UKR RES (76271820,76271748)",NaN,224
1,1129813,448,2023-08-31,2023-09-14,Only tickets,RZESZOW-ANTALYA ( Only Ticket ),NaN,No,NaN,Warszawa,...,448,0,0,448,EUR,p_anna,p_anna,RT UKR RES 76268649,[Res.No:81129813],448
2,1129807,3584,2023-08-31,2023-09-07,Only tickets,RZESZOW-ANTALYA ( Only Ticket ),NaN,No,NaN,Warszawa,...,3584,0,0,3584,EUR,p_anna,p_anna,"RT UKR RES (76272737,76272584,76272191,7627116...",NaN,3584
3,1129803,6944,2023-08-31,2023-09-10,Only tickets,RZESZOW-ANTALYA ( Only Ticket ),NaN,No,NaN,Warszawa,...,6944,0,0,6944,EUR,p_anna,p_anna,"RT UKR RES (76272845,76272675,76272595,7627254...",[Res.No:81129803],6944
4,1129782,672,2023-08-27,2023-09-10,Only tickets,RZESZOW-ANTALYA ( Only Ticket ),NaN,No,NaN,Warszawa,...,672,0,0,672,EUR,p_anna,p_anna,RT UKR RES (76269555),[Res.No:81129782],672


In [6]:
dmcc_pln = pd.read_excel('dmcc.xlsx', sheet_name='Samo w PLN', header=0)
dmcc_pln['res'] = dmcc_pln['res'].astype(str)

In [7]:
dmcc_pln.head()

,res,Catalogue price SAMO PLN
0,1134992,52670
1,1140530,41440
2,1123026,38080
3,1148278,38080
4,1136733,37785


In [8]:
result = pd.merge(dmcc_sergan, dmcc_pln, left_on='Reservation', right_on='res', how='outer')

In [13]:
# merge the columns 'Reservation' and 'res' into one column
result['res'] = result['Reservation'].fillna(result['res'])

In [15]:
# save to the same excel file but under sheet 'Podsum' 
with pd.ExcelWriter('dmcc.xlsx', mode='a') as writer:  
    result.to_excel(writer, sheet_name='Podsum', index=False)

In [ ]:
result